# DCGAN Implementation
- 使用image->mnist image
- 使用random vector
- 输出：img(类似于mnist中的)

In [6]:
"""
1 Data provider
    a.img data
    b.random vector data
2 build calculate graph
    a.gnerator
    b.discriminator
    c.DCGAN(combine a. b.)
3 training process
"""
import os
import sys
import pickle
import tensorflow as tf
from tensorflow import logging
from tensorflow import gfile
import numpy as np
import pprint
import math
import random
from PIL import Image
from tensorflow.examples.tutorials.mnist import input_data

output_dir = './local_run'
mnist_dir = './MNIST_DATA/'
'''
network 原因无法自动下载导入 手动创建目录导入
mnist = input_data.read_data_sets(mnist_dir,one_hot=True)
'''
if not gfile.Exists(mnist_dir):
    gfile.Exists(mnist_dir)
mnist = input_data.read_data_sets(mnist_dir,one_hot=True)    
if not gfile.Exists(output_dir):
    gfile.MakeDirs(output_dir)


Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./MNIST_DATA/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./MNIST_DATA/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting ./MNIST_DATA/t10k-images-idx3-ubyte.gz
Extracting ./MNIST_DATA/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [9]:
def get_default_params():
    return tf.contrib.training.HParams(
        v_dim = 100,
        init_conv_size = 4,
        g_channels = [128,64,32,1],
        d_channels = [32,64,128,256],
        batch_size = 128,
        learning_rate = 1e-3,
        beta1 = 0.9,
        img_size = 32,#生成img size 32*32 ！= 784
    )
hps = get_default_params()
print(hps.init_conv_size)
# 28*28 = 784
print(mnist.train.images.shape)
print(mnist.test.images.shape)
print(mnist.train.labels.shape)
print(mnist.test.labels.shape)

4
(55000, 784)
(10000, 784)
(55000, 10)
(10000, 10)


In [17]:
class Mnist_DCGANDataSet:
    """
    对于mnist提供数据封装 同时提供random vector
    提供next_batch 的 random vector + mnist img
    """
    def __init__(self,mnist_img_data,v_dim,img_size,need_shuffle):
        self._data = mnist_img_data
        self._data_size = len(self._data)
        self._v = np.random.standard_normal((self._data_size,v_dim))
        
        self._need_shuffle = need_shuffle
        self._indicator = 0
        '''img size 32*32 ！= 784 更新self._data'''
        self._resize_mnist_img(img_size)
        self._random_shuffle()
        
    def _random_shuffle(self):
        idx = np.random.permutation(self._data_size)
        '''只有np下的array才可以使用这种索引方式改变数据排列'''
        self._v = self._v[idx]
        self._data = self._data[idx]
    
    def _resize_mnist_img(self,img_size):
        """read mnist data and resize mnist img to goal size
        1 numpy -> PIL Image
        2 PIL Image resize
        3 PIL Image -> numpy 
        """
        data = np.asarray(self._data*255,np.uint8)
        # 784 - > 28*28
        data = data.reshape((self._data_size,28,28))
        resized_data = []
        for i in range(self._data_size):
            img = data[i]
            img = Image.fromarray(img)
            img = img.resize((img_size,img_size))
            img = np.asarray(img)
            # 需要通道信息 mnist单通道
            img = img.reshape((img_size,img_size,1))
            resized_data.append(img)
        # ->(datasize,img_size,img_size,1)
        resized_data = np.asarray(resized_data,np.float32)
        resized_data = resized_data / 127.5 - 1
        self._data = resized_data
        
    def next_batch(self,batch_size):
        if batch_size > self._data_size:
            raise Exception('batch_size is too large')
        end_indicator = self._indicator + batch_size
        if end_indicator > self._data_size:
            if self._need_shuffle:
                self._random_shuffle()
            self._indicator = 0
            end_indicator = batch_size
        batch_data = self._data[self._indicator:end_indicator]
        batch_v = self._v[self._indicator:end_indicator]
        self._indicator = end_indicator
        return batch_data,batch_v
    
mnist_dataset = Mnist_DCGANDataSet(mnist.train.images,
                                  hps.v_dim,hps.img_size,True)
batch_data,batch_v = mnist_dataset.next_batch(1)

'''
img = Image.fromarray(np.asarray((batch_data.reshape((32,32))+1)*127.5,
                      np.uint8))
img.show()    
'''
print(batch_data.shape)
print(batch_v.shape)

(1, 32, 32, 1)
(1, 100)


In [18]:
'''calculate graph construct'''
'''生成器'''
def conv2d_transpose(inputs,out_channel,name,
                    training,with_bn_relu=True):
    """
    由于DCGAN中bn在初始层结束层没有 所以需要选择 
    实现对于完整的反卷积层封装
    """
    with tf.variable_scope(name):
        conv2d_trans = tf.layers.conv2d_transpose(inputs,
                                                 out_channel,
                                                 [5,5],
                                                 strides=(2,2),
                                                 padding='SAME')
        if with_bn_relu:
            bn = tf.layers.batch_normalization(conv2d_trans,
                                              training=training)
            return tf.nn.relu(bn)
        else:
            return conv2d_trans

class Generator:
    """生成器定义 借助反卷积"""
    def __init__(self,channels,init_conv_size):
        self._channels = channels
        self._init_conv_size = init_conv_size
        self._reuse = False
        
    def __call__(self,inputs,training):
        """魔法方法：实现对象像函数一样调用"""
        # 保证无论是placeholder 还是 np数组都变成tensor
        inputs = tf.convert_to_tensor(inputs)
        with tf.variable_scope('generator',reuse=self._reuse):
            """
            random v-(fc)->channel[0]*init_conv_size**2->reshape
            """
            with tf.variable_scope('input2img_fc'):
                fc = tf.layers.dense(inputs,
                                    self._channels[0]*self._init_conv_size**2)
                conv0 = tf.reshape(fc,
                                   [-1,self._init_conv_size,
                                    self._init_conv_size,
                                    self._channels[0]])
                bn0 = tf.layers.batch_normalization(conv0,training=training)
                relu0 = tf.nn.relu(bn0)
            
            deconv_inputs = relu0
            # 搭建反卷积层
            for i in range(1,len(self._channels)):
                with_bn_relu = (i!=len(self._channels)-1)
                deconv_inputs = conv2d_transpose(deconv_inputs,
                                                self._channels[i],
                                                'deconv-%d'%i,
                                                training,
                                                with_bn_relu)
                img_inputs = deconv_inputs
                with tf.variable_scope('genetate_image'):
                    imgs = tf.tanh(img_inputs)
                    
        self._reuse = True
        '''成员变量保存所有生成器参数 GAN中分别训练所有变量'''
        # 铜鼓get_collection 获取制定scope中的一些量 比如变量等 通过key关键字制定 
        # 关键字保存在tf.GraphKeys....
        self.variables = tf.get_collection(key=tf.GraphKeys.TRAINABLE_VARIABLES,
                                          scope='generator')
        return imgs
    
        

In [25]:
'''判别器'''
def conv2d(inputs,out_channel,name,training,leak=0.1):
    """统一经过bn 对于conv2d封装 这里使用leaky relu"""
    def leaky_relu(x,leak=0.01,name ='leaky_relu'):
        #单纯比较大下可以使用maximum
        '''
        tf.maximum(a,b) 返回a,b之间的最大值
        tf.argmax(a,b) 返回在b维上a的最大值
        tf.reduce_max(a) 返回在张量a上的最大值
        '''
        return tf.maximum(x,x*leak,name=name)
    
    with tf.variable_scope(name):
        conv2d = tf.layers.conv2d(inputs,out_channel,
                                 [5,5],strides=(2,2),
                                 padding='SAME')
        bn = tf.layers.batch_normalization(conv2d,training=training)
        return leaky_relu(bn,leak=leak)

class Discriminator:
    def __init__(self,channels):
        self._channels = channels
        self._reuse = False
    def __call__(self,inputs,training):
        conv_inputs = tf.convert_to_tensor(inputs)
        with tf.variable_scope('discriminator',reuse=self._reuse):
            for i in range(len(self._channels)):
                conv_inputs = conv2d(conv_inputs,self._channels[i],
                                    'conv-%d'%i,training,0.1)
            fc_inputs = conv_inputs
            with tf.variable_scope('fc'):
                flatten = tf.layers.flatten(fc_inputs)
                # 生成器只需呀判断真假 所以输出为1/2 均可
                logits = tf.layers.dense(flatten,2,name='logits')
        self._reuse = True
        self.variables = tf.get_collection(key=tf.GraphKeys.TRAINABLE_VARIABLES,
                                          scope='discriminator')
        return logits           

In [30]:
'''定义DCGAN combine gen and dis'''
class DCGAN:
    """build structure and build ops"""
    def __init__(self,hps):
        self._g_channels = hps.g_channels
        self._d_channels = hps.d_channels
        self._batch_size = hps.batch_size
        self._init_conv_size = hps.init_conv_size
        self._v_dim = hps.v_dim
        self._img_size = hps.img_size
        self._learning_rate = hps.learning_rate
        self._beta1 = hps.beta1
        
        self._generator = Generator(self._g_channels,self._init_conv_size)
        self._discriminator = Discriminator(self._d_channels)
        
    def build(self):
        """build structure and return placeholder loss acc等"""
        self._v_placeholder = tf.placeholder(
            tf.float32,(self._batch_size,self._v_dim))
        self._img_placeholder = tf.placeholder(
            tf.float32,(self._batch_size,self._img_size,self._img_size,1))
        # output
        self._generated_img = self._generator(self._v_placeholder,True)
        '''
        需要使用同一网络结构进行两次输入运算 而不是每个输入对应一个网络结构
        所以需要用reuse 制定 网络结构重用
        '''
        fake_img_logits = self._discriminator(self._generated_img,True)
        real_img_logits = self._discriminator(self._img_placeholder,True)
        # loss **对于原有数学公式进行拆分 得到用tf自带函数实现的方式 技巧**
        '''
        Value passed to parameter 'labels' has DataType int8 
        not in list of allowed values: int32, int64
        '''
        loss_on_fake_to_real = tf.reduce_mean(
            tf.nn.sparse_softmax_cross_entropy_with_logits(
                labels = tf.ones([self._batch_size],tf.int32),
                logits = fake_img_logits))
        
        loss_on_real_to_real = tf.reduce_mean(
            tf.nn.sparse_softmax_cross_entropy_with_logits(
                labels = tf.ones([self._batch_size],tf.int32),
                logits = real_img_logits))
        
        loss_on_fake_to_fake = tf.reduce_mean(
            tf.nn.sparse_softmax_cross_entropy_with_logits(
                labels = tf.zeros([self._batch_size],tf.int32),
                logits = fake_img_logits))
        
        '''以下是新的tf中的管理变量的方式 使用collection'''
        # 保存
        tf.add_to_collection('g_losses',loss_on_fake_to_real)
        tf.add_to_collection('d_losses',loss_on_real_to_real)
        tf.add_to_collection('d_losses',loss_on_fake_to_fake)
        # 提取
        self._loss = {
            'gen': tf.add_n(tf.get_collection('g_losses'),name='total_g_loss'),
            'disc':tf.add_n(tf.get_collection('d_losses'),name='total_d_loss')
        }
        
        return self._v_placeholder,self._img_placeholder,self._generated_img,self._loss
    
    def build_op(self):
        """return builded operation
        must be called after build function
        """
        g_optimizer = tf.train.AdamOptimizer(self._learning_rate,self._beta1)
        d_optimizer = tf.train.AdamOptimizer(self._learning_rate,self._beta1)
        '''分别执行不同的操作 借助gen disc中的variables'''
        # apply_gradient minimize 等方法都可以应用在指定的变量上 很灵活
        g_opt_op = g_optimizer.minimize(self._loss['gen'],
                              var_list = self._generator.variables)
        d_opt_op = d_optimizer.minimize(self._loss['disc'],
                              var_list = self._discriminator.variables)
        
        '''
        难点 控制而知运行顺序 使用tf高级操作 control_dependencies 
        中的操作在列表中操作执行后才会执行
        no_op空操作 占位符存在
        '''
        with tf.control_dependencies([g_opt_op,d_opt_op]):
            return tf.no_op(name='train_op')

tf.reset_default_graph()
dcgan = DCGAN(hps)
v_placeholder,img_placeholder,generated_imgs,loss = dcgan.build()
train_op = dcgan.build_op()

In [34]:
'''train process'''
def combine_graph(batch_imgs,img_size,rows=8,cols=16):
    """combine small imgs in a batch into a big img to show
    Args:
    - rows*clos = batch_size
    """
    result_big_img = []
    for i in range(rows):
        row_imgs = []
        for j in range(cols):
            img = batch_imgs[i*cols+j]
            img = img.reshape((img_size,img_size))
            img = (img+1)*127.5
            row_imgs.append(img)
        '''hstack vstack减少一个维度'''
        row_imgs = np.hstack(row_imgs)
        result_big_img.append(row_imgs)
    result_big_img = np.vstack(result_big_img)
    result_big_img = np.asarray(result_big_img,np.uint8)
    result_big_img = Image.fromarray(result_big_img)
    return result_big_img

init_op = tf.global_variables_initializer()
train_steps = 10000
sample_steps = 200

with tf.Session() as sess:
    sess.run(init_op)
    for i in range(train_steps):
        need_sample = ((i+1)%sample_steps == 0)
        batch_imgs,batch_v = mnist_dataset.next_batch(hps.batch_size)
        fetches = [train_op,loss['gen'],loss['disc']]
        
        if need_sample:
            fetches.append(generated_imgs)
        outputs = sess.run(fetches,feed_dict={
                            v_placeholder:batch_v,
                            img_placeholder:batch_imgs})
        
        _,g_loss_val,d_loss_val = outputs[0:3]
        logging.info('step:%4d,g_loss:%4.3f,d_loss:%4.3f'
                     %(i,g_loss_val,d_loss_val))
        if need_sample:
            generated_imgs_val = outputs[-1]
            gen_img_path = os.path.join(output_dir,'%04d-gen.jpg'%(i+1))
            real_img_path = os.path.join(output_dir,'%04d-real.jpg'%(i+1))
            
            generated_img = combine_graph(generated_imgs_val,hps.img_size)
            real_img = combine_graph(batch_imgs,hps.img_size)
            # Image 对象调用保存函数
            generated_img.save(gen_img_path)
            real_img.save(real_img_path)

INFO:tensorflow:step:   0,g_loss:0.979,d_loss:1.641
INFO:tensorflow:step:   1,g_loss:1.001,d_loss:0.951
INFO:tensorflow:step:   2,g_loss:16.916,d_loss:0.002
INFO:tensorflow:step:   3,g_loss:18.290,d_loss:0.348
INFO:tensorflow:step:   4,g_loss:13.416,d_loss:0.029
INFO:tensorflow:step:   5,g_loss:0.677,d_loss:2.608
INFO:tensorflow:step:   6,g_loss:17.501,d_loss:0.067
INFO:tensorflow:step:   7,g_loss:20.445,d_loss:0.997
INFO:tensorflow:step:   8,g_loss:18.123,d_loss:0.115
INFO:tensorflow:step:   9,g_loss:11.285,d_loss:0.032
INFO:tensorflow:step:  10,g_loss:0.583,d_loss:3.028
INFO:tensorflow:step:  11,g_loss:13.094,d_loss:0.001
INFO:tensorflow:step:  12,g_loss:17.781,d_loss:0.069
INFO:tensorflow:step:  13,g_loss:19.219,d_loss:0.076
INFO:tensorflow:step:  14,g_loss:19.378,d_loss:0.479
INFO:tensorflow:step:  15,g_loss:17.565,d_loss:0.240
INFO:tensorflow:step:  16,g_loss:13.000,d_loss:0.112
INFO:tensorflow:step:  17,g_loss:5.579,d_loss:0.049
INFO:tensorflow:step:  18,g_loss:0.062,d_loss:4.315

INFO:tensorflow:step: 156,g_loss:11.146,d_loss:0.027
INFO:tensorflow:step: 157,g_loss:9.207,d_loss:0.002
INFO:tensorflow:step: 158,g_loss:6.663,d_loss:0.012
INFO:tensorflow:step: 159,g_loss:1.510,d_loss:1.046
INFO:tensorflow:step: 160,g_loss:2.520,d_loss:0.261
INFO:tensorflow:step: 161,g_loss:5.839,d_loss:0.015
INFO:tensorflow:step: 162,g_loss:6.707,d_loss:0.047
INFO:tensorflow:step: 163,g_loss:6.959,d_loss:0.054
INFO:tensorflow:step: 164,g_loss:6.303,d_loss:0.324
INFO:tensorflow:step: 165,g_loss:5.012,d_loss:0.484
INFO:tensorflow:step: 166,g_loss:2.371,d_loss:0.403
INFO:tensorflow:step: 167,g_loss:2.957,d_loss:0.278
INFO:tensorflow:step: 168,g_loss:5.227,d_loss:0.050
INFO:tensorflow:step: 169,g_loss:6.247,d_loss:0.016
INFO:tensorflow:step: 170,g_loss:6.732,d_loss:0.013
INFO:tensorflow:step: 171,g_loss:7.401,d_loss:0.011
INFO:tensorflow:step: 172,g_loss:7.770,d_loss:0.008
INFO:tensorflow:step: 173,g_loss:7.991,d_loss:0.007
INFO:tensorflow:step: 174,g_loss:7.783,d_loss:0.009
INFO:tensor

INFO:tensorflow:step: 314,g_loss:6.321,d_loss:0.006
INFO:tensorflow:step: 315,g_loss:5.597,d_loss:0.008
INFO:tensorflow:step: 316,g_loss:5.138,d_loss:0.015
INFO:tensorflow:step: 317,g_loss:6.354,d_loss:0.005
INFO:tensorflow:step: 318,g_loss:7.433,d_loss:0.007
INFO:tensorflow:step: 319,g_loss:7.933,d_loss:0.011
INFO:tensorflow:step: 320,g_loss:8.034,d_loss:0.002
INFO:tensorflow:step: 321,g_loss:8.217,d_loss:0.013
INFO:tensorflow:step: 322,g_loss:7.431,d_loss:0.003
INFO:tensorflow:step: 323,g_loss:7.064,d_loss:0.011
INFO:tensorflow:step: 324,g_loss:6.733,d_loss:0.005
INFO:tensorflow:step: 325,g_loss:6.387,d_loss:0.007
INFO:tensorflow:step: 326,g_loss:7.330,d_loss:0.004
INFO:tensorflow:step: 327,g_loss:7.284,d_loss:0.002
INFO:tensorflow:step: 328,g_loss:7.581,d_loss:0.002
INFO:tensorflow:step: 329,g_loss:8.073,d_loss:0.001
INFO:tensorflow:step: 330,g_loss:7.784,d_loss:0.002
INFO:tensorflow:step: 331,g_loss:8.157,d_loss:0.002
INFO:tensorflow:step: 332,g_loss:7.720,d_loss:0.002
INFO:tensorf

INFO:tensorflow:step: 472,g_loss:6.917,d_loss:0.002
INFO:tensorflow:step: 473,g_loss:7.032,d_loss:0.002
INFO:tensorflow:step: 474,g_loss:6.903,d_loss:0.002
INFO:tensorflow:step: 475,g_loss:7.062,d_loss:0.004
INFO:tensorflow:step: 476,g_loss:6.934,d_loss:0.002
INFO:tensorflow:step: 477,g_loss:7.015,d_loss:0.003
INFO:tensorflow:step: 478,g_loss:6.852,d_loss:0.004
INFO:tensorflow:step: 479,g_loss:6.925,d_loss:0.003
INFO:tensorflow:step: 480,g_loss:6.800,d_loss:0.003
INFO:tensorflow:step: 481,g_loss:6.604,d_loss:0.003
INFO:tensorflow:step: 482,g_loss:6.423,d_loss:0.003
INFO:tensorflow:step: 483,g_loss:6.414,d_loss:0.005
INFO:tensorflow:step: 484,g_loss:6.280,d_loss:0.004
INFO:tensorflow:step: 485,g_loss:5.908,d_loss:0.004
INFO:tensorflow:step: 486,g_loss:5.953,d_loss:0.004
INFO:tensorflow:step: 487,g_loss:5.911,d_loss:0.007
INFO:tensorflow:step: 488,g_loss:6.181,d_loss:0.004
INFO:tensorflow:step: 489,g_loss:6.051,d_loss:0.006
INFO:tensorflow:step: 490,g_loss:6.504,d_loss:0.003
INFO:tensorf

INFO:tensorflow:step: 630,g_loss:5.014,d_loss:0.045
INFO:tensorflow:step: 631,g_loss:4.327,d_loss:0.045
INFO:tensorflow:step: 632,g_loss:5.062,d_loss:0.024
INFO:tensorflow:step: 633,g_loss:5.223,d_loss:0.031
INFO:tensorflow:step: 634,g_loss:5.521,d_loss:0.021
INFO:tensorflow:step: 635,g_loss:5.509,d_loss:0.078
INFO:tensorflow:step: 636,g_loss:5.549,d_loss:0.037
INFO:tensorflow:step: 637,g_loss:4.804,d_loss:0.016
INFO:tensorflow:step: 638,g_loss:4.631,d_loss:0.016
INFO:tensorflow:step: 639,g_loss:5.574,d_loss:0.007
INFO:tensorflow:step: 640,g_loss:6.530,d_loss:0.003
INFO:tensorflow:step: 641,g_loss:7.084,d_loss:0.002
INFO:tensorflow:step: 642,g_loss:6.101,d_loss:0.010
INFO:tensorflow:step: 643,g_loss:5.611,d_loss:0.008
INFO:tensorflow:step: 644,g_loss:5.458,d_loss:0.009
INFO:tensorflow:step: 645,g_loss:4.712,d_loss:0.018
INFO:tensorflow:step: 646,g_loss:5.700,d_loss:0.007
INFO:tensorflow:step: 647,g_loss:6.353,d_loss:0.005
INFO:tensorflow:step: 648,g_loss:7.032,d_loss:0.015
INFO:tensorf

...

INFO:tensorflow:step:9266,g_loss:5.043,d_loss:0.030
INFO:tensorflow:step:9267,g_loss:3.407,d_loss:0.074
INFO:tensorflow:step:9268,g_loss:3.297,d_loss:0.078
INFO:tensorflow:step:9269,g_loss:7.262,d_loss:0.008
INFO:tensorflow:step:9270,g_loss:5.726,d_loss:0.033
INFO:tensorflow:step:9271,g_loss:8.085,d_loss:0.009
INFO:tensorflow:step:9272,g_loss:8.480,d_loss:0.040
INFO:tensorflow:step:9273,g_loss:6.767,d_loss:0.010
INFO:tensorflow:step:9274,g_loss:8.076,d_loss:0.072
INFO:tensorflow:step:9275,g_loss:5.832,d_loss:0.010
INFO:tensorflow:step:9276,g_loss:4.529,d_loss:0.043
INFO:tensorflow:step:9277,g_loss:5.106,d_loss:0.028
INFO:tensorflow:step:9278,g_loss:4.241,d_loss:0.039
INFO:tensorflow:step:9279,g_loss:4.817,d_loss:0.050
INFO:tensorflow:step:9280,g_loss:8.290,d_loss:0.001
INFO:tensorflow:step:9281,g_loss:7.176,d_loss:0.009
INFO:tensorflow:step:9282,g_loss:2.836,d_loss:0.258
INFO:tensorflow:step:9283,g_loss:3.998,d_loss:0.040
INFO:tensorflow:step:9284,g_loss:5.118,d_loss:0.013
INFO:tensorf

INFO:tensorflow:step:9422,g_loss:4.105,d_loss:0.040
INFO:tensorflow:step:9423,g_loss:6.215,d_loss:0.016
INFO:tensorflow:step:9424,g_loss:3.363,d_loss:0.077
INFO:tensorflow:step:9425,g_loss:5.642,d_loss:0.038
INFO:tensorflow:step:9426,g_loss:4.829,d_loss:0.270
INFO:tensorflow:step:9427,g_loss:3.035,d_loss:0.163
INFO:tensorflow:step:9428,g_loss:3.287,d_loss:0.084
INFO:tensorflow:step:9429,g_loss:3.879,d_loss:0.068
INFO:tensorflow:step:9430,g_loss:3.145,d_loss:0.097
INFO:tensorflow:step:9431,g_loss:5.090,d_loss:0.027
INFO:tensorflow:step:9432,g_loss:4.073,d_loss:0.097
INFO:tensorflow:step:9433,g_loss:3.663,d_loss:0.091
INFO:tensorflow:step:9434,g_loss:3.419,d_loss:0.107
INFO:tensorflow:step:9435,g_loss:4.516,d_loss:0.031
INFO:tensorflow:step:9436,g_loss:6.791,d_loss:0.438
INFO:tensorflow:step:9437,g_loss:3.576,d_loss:0.129
INFO:tensorflow:step:9438,g_loss:0.973,d_loss:0.830
INFO:tensorflow:step:9439,g_loss:5.063,d_loss:0.019
INFO:tensorflow:step:9440,g_loss:5.919,d_loss:0.062
INFO:tensorf

INFO:tensorflow:step:9580,g_loss:7.993,d_loss:0.138
INFO:tensorflow:step:9581,g_loss:7.676,d_loss:1.002
INFO:tensorflow:step:9582,g_loss:5.337,d_loss:0.030
INFO:tensorflow:step:9583,g_loss:3.462,d_loss:0.078
INFO:tensorflow:step:9584,g_loss:0.815,d_loss:0.918
INFO:tensorflow:step:9585,g_loss:3.129,d_loss:0.086
INFO:tensorflow:step:9586,g_loss:2.625,d_loss:0.147
INFO:tensorflow:step:9587,g_loss:2.691,d_loss:0.139
INFO:tensorflow:step:9588,g_loss:4.238,d_loss:0.043
INFO:tensorflow:step:9589,g_loss:8.121,d_loss:0.061
INFO:tensorflow:step:9590,g_loss:7.116,d_loss:0.318
INFO:tensorflow:step:9591,g_loss:7.587,d_loss:2.512
INFO:tensorflow:step:9592,g_loss:2.521,d_loss:0.166
INFO:tensorflow:step:9593,g_loss:2.055,d_loss:0.255
INFO:tensorflow:step:9594,g_loss:0.490,d_loss:1.309
INFO:tensorflow:step:9595,g_loss:0.797,d_loss:0.842
INFO:tensorflow:step:9596,g_loss:9.391,d_loss:0.013
INFO:tensorflow:step:9597,g_loss:9.657,d_loss:0.066
INFO:tensorflow:step:9598,g_loss:12.100,d_loss:1.405
INFO:tensor

INFO:tensorflow:step:9738,g_loss:3.696,d_loss:0.068
INFO:tensorflow:step:9739,g_loss:2.465,d_loss:0.204
INFO:tensorflow:step:9740,g_loss:3.415,d_loss:0.080
INFO:tensorflow:step:9741,g_loss:1.843,d_loss:0.330
INFO:tensorflow:step:9742,g_loss:2.312,d_loss:0.173
INFO:tensorflow:step:9743,g_loss:6.054,d_loss:0.005
INFO:tensorflow:step:9744,g_loss:6.223,d_loss:0.007
INFO:tensorflow:step:9745,g_loss:11.788,d_loss:0.017
INFO:tensorflow:step:9746,g_loss:11.618,d_loss:0.127
INFO:tensorflow:step:9747,g_loss:10.705,d_loss:0.187
INFO:tensorflow:step:9748,g_loss:10.588,d_loss:0.111
INFO:tensorflow:step:9749,g_loss:11.407,d_loss:0.025
INFO:tensorflow:step:9750,g_loss:10.502,d_loss:0.066
INFO:tensorflow:step:9751,g_loss:11.498,d_loss:0.011
INFO:tensorflow:step:9752,g_loss:5.967,d_loss:0.024
INFO:tensorflow:step:9753,g_loss:8.705,d_loss:0.001
INFO:tensorflow:step:9754,g_loss:8.348,d_loss:0.002
INFO:tensorflow:step:9755,g_loss:6.247,d_loss:0.008
INFO:tensorflow:step:9756,g_loss:6.633,d_loss:0.009
INFO:

INFO:tensorflow:step:9896,g_loss:7.476,d_loss:0.329
INFO:tensorflow:step:9897,g_loss:5.471,d_loss:0.335
INFO:tensorflow:step:9898,g_loss:4.266,d_loss:0.045
INFO:tensorflow:step:9899,g_loss:1.755,d_loss:0.376
INFO:tensorflow:step:9900,g_loss:1.294,d_loss:0.571
INFO:tensorflow:step:9901,g_loss:6.621,d_loss:0.092
INFO:tensorflow:step:9902,g_loss:8.691,d_loss:0.313
INFO:tensorflow:step:9903,g_loss:8.820,d_loss:0.071
INFO:tensorflow:step:9904,g_loss:8.911,d_loss:0.399
INFO:tensorflow:step:9905,g_loss:10.262,d_loss:0.283
INFO:tensorflow:step:9906,g_loss:4.770,d_loss:0.025
INFO:tensorflow:step:9907,g_loss:0.319,d_loss:1.866
INFO:tensorflow:step:9908,g_loss:4.746,d_loss:0.026
INFO:tensorflow:step:9909,g_loss:7.308,d_loss:0.011
INFO:tensorflow:step:9910,g_loss:10.056,d_loss:0.076
INFO:tensorflow:step:9911,g_loss:13.673,d_loss:0.848
INFO:tensorflow:step:9912,g_loss:10.300,d_loss:1.244
INFO:tensorflow:step:9913,g_loss:6.284,d_loss:0.004
INFO:tensorflow:step:9914,g_loss:1.838,d_loss:0.335
INFO:ten

# 问题：生成器学的很差 辨别器太强 对于这种情况如何调优？？